In [54]:
#Basic imports
import pandas as pd
import numpy as np
import re
from os import listdir
from os.path import isfile, join

# Tracking msg 

In [55]:
#Open files
trackFile = open('/home/sabrine/notebook/messages/trackMe.txt', 'r') 

In [56]:
# Split text file in order to extract different field
track = trackFile.read().split('header: \n')[1:]

In [57]:
# Create emtpy Series that will become the columns f the final dataframe
seq = []
secs = []
nsecs = []
tracks = []

# Fill the Serie using appropriate regex to extract info
for i in range(len(track)):
    seq.append(re.search('[0-9]+',re.search('seq: [0-9]+', track[i]).group(0)).group(0))        
    secs.append(re.search('[0-9]+',re.search('stamp: \n    secs: [0-9]+', track[i]).group(0)).group(0))
    nsecs.append(re.search('[0-9]+',re.search('nsecs:\s+[0-9]+', track[i]).group(0)).group(0))
    tracks.append(re.search('tracks:(?s).*', track[i]).group().replace('tracks: ', ''))

In [58]:
# Same as search function of regex, except return a nan object if regex not found
def advSearch(reg, string):
    if isinstance(string, basestring) and re.search(reg, string):
        return re.search(reg, string).group(0)
    else:
        return np.nan

In [59]:
# Process thet tracking information
# Create different Series for the different information to easely process then later
id_ = []
height = []
distance = []
x_bb = []
y_bb = []
width_bb = []
height_bb = []

# Fill the Serie using appropriate regex to extract info
for trackingInfo in tracks:
    id_.append(advSearch('[0-9]+',advSearch('id: [0-9]+\n', trackingInfo)))
    height.append(advSearch('[0-9]+(\.[0-9]+)?',advSearch('height: [0-9]+(\.[0-9]+)?', trackingInfo)))
    distance.append(advSearch('[0-9]+(\.[0-9]+)?', advSearch('distance: [0-9]+(\.[0-9]+)?', trackingInfo)))
    x_bb.append(advSearch('[0-9]+(\.[0-9]+)?',advSearch('x: [0-9]+(\.[0-9]+)?', trackingInfo)))
    y_bb.append(advSearch('[0-9]+(\.[0-9]+)?',advSearch('y: [0-9]+(\.[0-9]+)?', trackingInfo)))
    width_bb.append(advSearch('[0-9]+\n',advSearch('width: [0-9]+\n', trackingInfo)))
    height_bb.append(advSearch('[0-9]+\n',advSearch('height: [0-9]+\n', trackingInfo)))

In [60]:
tra = pd.DataFrame(columns=['seq', 'secs', 'nsecs', 'tracks','id_', 'height', 'distance',
                            'x_bb', 'y_bb', 'width_bb', 'height_bb', 'time', 'frameMatch'])

In [61]:
tra.seq = seq
tra.secs = secs
tra.nsecs = nsecs
tra.tracks = tracks
tra.id_ = id_
tra.height = height
tra.distance = distance
tra.x_bb = x_bb
tra.y_bb = y_bb
tra.width_bb = width_bb
tra.height_bb = height_bb

In [62]:
# Recreate the time of the detection in order to compare it to the frames' file name
timeDetec = (tra.secs + tra.nsecs).apply(pd.to_numeric).values.T.tolist()
tra.time = timeDetec

In [63]:
tra.head()

,seq,secs,nsecs,tracks,id_,height,distance,x_bb,y_bb,width_bb,height_bb,time,frameMatch
0,2247,1493379743,701172000,[]\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493379743701172000,NaN
1,2248,1493379743,755150000,[]\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493379743755150000,NaN
2,2249,1493379743,809035000,[]\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493379743809035000,NaN
3,2250,1493379743,884731000,[]\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493379743884731000,NaN
4,2251,1493379743,939791000,[]\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493379743939791000,NaN


# RGB images

In [64]:
# List all file from frames folder
mypath = "/home/sabrine/notebook/framesRGB"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [65]:
# Extract time from files' name
time_ = []
for t in onlyfiles:
    time_.append(int(re.search('^[0-9]*', t).group()))

# Synchronization

In [66]:
tra.frameMatch = time_[:-2]

In [67]:
tra.head()

,seq,secs,nsecs,tracks,id_,height,distance,x_bb,y_bb,width_bb,height_bb,time,frameMatch
0,2247,1493379743,701172000,[]\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493379743701172000,1493379751039782385
1,2248,1493379743,755150000,[]\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493379743755150000,1493379753361718849
2,2249,1493379743,809035000,[]\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493379743809035000,1493379750271873950
3,2250,1493379743,884731000,[]\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493379743884731000,1493379758271101078
4,2251,1493379743,939791000,[]\n \n,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1493379743939791000,1493379751634413259


In [68]:
tra.to_pickle("/home/sabrine/social_tracking/syncPresentation")